In [135]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import shutil

In [150]:
labels_path = "/home/filiptkac/Downloads/train.csv"
images_path = "/home/filiptkac/Downloads/abd-trauma-dataset"
images_path_reduced = "/home/filiptkac/Downloads/abd-trauma-dataset-reduced"

In [24]:
patients = os.listdir(images_path)

min_images = None
max_images = None
reduced_images_number = 100
patients_too_few_images = []

for patient in patients:
    files = os.listdir(f"{images_path}/{patient}")
    files_amount = len(files)
    if min_images is None or files_amount < min_images:
        min_images = files_amount
    if max_images is None or files_amount > max_images:
        max_images = files_amount
    if files_amount < reduced_images_number:
        patients_too_few_images.append(patient)

print(f"minimum number of images in CT: {min_images}, maximum number of images in CT: {max_images}")
print(f"number of CT scans with less than {reduced_images_number} images: {len(patients_too_few_images)}")

minimum number of images in CT: 46, maximum number of images in CT: 1727
number of CT scans with less than hundred images: 261


In [137]:
if not os.path.exists(images_path_reduced):
    os.makedirs(images_path_reduced)
    
for patient in tqdm(patients):
    if patient in patients_too_few_images:
        continue
        
    files = os.listdir(f"{images_path}/{patient}")
    files_indices = np.int32(np.linspace(0, len(files) - 1, reduced_images_number))
    files = list(map(lambda fname: os.path.splitext(fname)[0], files))
    files = sorted([int(x) for x in files])
    files = np.array(files)
    files = files[files_indices]
    files = [f"{file}.png" for file in files]
    
    new_patient_folder = f"{images_path_reduced}/{patient}"
    if not os.path.exists(new_patient_folder):
        os.makedirs(new_patient_folder)
        
    for file in files:
        shutil.copy(f"{images_path}/{patient}/{file}", f"{new_patient_folder}/{file}")

100%|██████████| 3147/3147 [02:33<00:00, 20.48it/s]


In [151]:
df = pd.read_csv(labels_path)
df.head(5)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1


In [153]:
print(df.shape[0])
patients_too_few_images_int = [int(pid) for pid in patients_too_few_images]
reduced_df = df[~df['patient_id'].isin(patients_too_few_images_int)]
print(reduced_df.shape[0])
reduced_df.to_csv("/home/filiptkac/Downloads/train_reduced.csv")

3147
2886


In [155]:
reduced_patient_ids = reduced_df['patient_id'].values

for pid in reduced_patient_ids:
    if not os.path.exists(f"{images_path_reduced}/{pid}"):
        print(f"{images_path_reduced}/{pid} does not exist!!")
        
print("folder analysis complete")

folder analysis complete


In [160]:
labels = reduced_df.iloc[0].drop('patient_id')
print(labels.values)

[1 0 0 1 0 1 0 1 0 0 0 0 1 1]
